## PLST - Rebalancing Data for April 2024

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase, MethodologyProd
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
date = '2024-04-01'
min_mcap =  0
min_weight = 0.005
max_weight = 0.15
circ_supply_threshold = 0
liveness_threshold = 30 # In days
liquidity_consistency = 90 # In days
coingecko_category = ["liquid-staking-tokens","liquid-restaking-tokens"]
index_address = '0x8b16626d12E7ef3B41712bB8Ff88B8E4e04c165B'

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyProd(
                                date,
                                2,
                                'arbitrum-one',
                                index_address,
                                'plsx_benchmark_data',
                                'plsx_liquidities',
                                min_mcap,
                                min_weight,
                                max_weight,
                                65535,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)
print(max_slippage)

-0.04


### Token inclusion criteria

1. **Project and token characteristics**
    1. The token should be listed on CoinGecko with pricing data spanning at least 1 month prior to the date of inclusion in the index.
    1. No rebasing tokens, the staking yield should accrue to the token's price.
    1. The token should be 100% collateralised by the underlying assets.
   <br>
1. **Pricing Requirements**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange on a supported chain.
    1. The price should experience no more than 5% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety.
1. **Weighting Requirements**
    1. Each token is weighted according to its market cap.
    1. The maximum weight any one token can have is 15%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the maximum weight the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results,liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens],
    ids_to_replace=[['staked-ether'],['wrapped-steth']],  
    remove_category_assets=['frax-ether','ankreth'],
    platforms_to_add = [{'id':'kelp-dao-restaked-eth','blockchain':'arbitrum-one','address':'0x4186BFC76E2E237523CBC30FD220FE055156b41F'}],
    platforms_to_remove=['ethereum'],   
    values_to_update=[{'id':'benqi-liquid-staked-avax','category':'symbol','value':"sAVAX"},
                      {'id':'rocket-pool-eth','category':'symbol','value':"rETH"},
                      {'id':'wrapped-steth','category':'symbol','value':"wstETH"},
                      {'id':'staked-frax-ether','category':'symbol','value':"sfrxETH"},
                      {'id':'stader-ethx','category':'symbol','value':"ETHx"},
                      {'id':'stakewise-v3-oseth','category':'symbol','value':"osETH"},
                      {'id':'restaked-swell-eth','category':'circulating_supply','value':50789.180445528569814269},
                      {'id':'restaked-swell-eth','category':'total_supply','value':50789.180445528569814269},
                      {'id':'restaked-swell-eth','category':'market_cap','value':50789.180445528569814269*3500},
                      {'id':'restaked-swell-eth','category':'symbol','value':'rswETH'},
                      {'id':'renzo-restaked-eth','category':'circulating_supply','value':359925.079372744947154147},
                      {'id':'renzo-restaked-eth','category':'total_supply','value':359925.079372744947154147},
                      {'id':'renzo-restaked-eth','category':'market_cap','value':359925.079372744947154147*3500},
                      {'id':'renzo-restaked-eth','category':'symbol','value':'ezETH'},
                      ],
    onchain_oracles=['coinbase-wrapped-staked-eth','lido-staked-matic','sweth','kelp-dao-restaked-eth','stader-maticx','wrapped-eeth']
    )
liquidity

/Users/olivermehr/Documents/GitHub/rebalancing-data/PLSX/../methodology.py:138: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.category_data = pd.concat(dataframe_list, axis=0)


Excluding dinero-staked-eth, pricing data available only for 28 < 30 days
Table created successfully
Table created successfully
mantle not supported
sweth on arbitrum-one returned an invalid API response
Manually calculating slippage for staked-frax-ether on ethereum
moonbeam not supported
staked-frax-ether on fantom returned an invalid API response
Manually calculating slippage for staked-frax-ether on optimistic-ethereum
Manually calculating slippage for staked-frax-ether on arbitrum-one
staked-frax-ether on polygon-pos returned an invalid API response
staked-frax-ether on binance-smart-chain returned an invalid API response
stader-maticx on ethereum returned an invalid API response
manta-pacific not supported
trufin-staked-matic on ethereum returned an invalid API response
Manually calculating slippage for trufin-staked-matic on polygon-pos
bridge-oracle on ethereum returned an invalid API response
bridge-oracle on arbitrum-one returned an invalid API response
nuls not supported
ank

,arbitrum-one,avalanche,base,binance-smart-chain,fantom,optimistic-ethereum,polygon-pos,optimal chain,best slippage,best slippage chain
id,,,,,,,,,,
wrapped-steth,-0.001651,NaN,NaN,NaN,NaN,-0.009720,-0.037323,arbitrum-one,-0.001651,arbitrum-one
rocket-pool-eth,-0.002084,NaN,-0.002470,NaN,NaN,-0.011154,-0.963252,arbitrum-one,-0.002084,arbitrum-one
renzo-restaked-eth,-0.002217,NaN,-0.049521,-0.265427,NaN,NaN,NaN,arbitrum-one,-0.002217,arbitrum-one
coinbase-wrapped-staked-eth,-0.007492,NaN,-0.002383,NaN,NaN,-0.999999,-0.847934,arbitrum-one,-0.002383,base
wrapped-eeth,-0.002401,NaN,NaN,NaN,NaN,NaN,NaN,arbitrum-one,-0.002401,arbitrum-one
staked-frax-ether,-0.002700,NaN,NaN,NaN,NaN,-0.010953,NaN,arbitrum-one,-0.002700,arbitrum-one
kelp-dao-restaked-eth,-0.002922,NaN,NaN,NaN,NaN,NaN,NaN,arbitrum-one,-0.002922,arbitrum-one
pstake-staked-bnb,NaN,NaN,NaN,-0.003276,NaN,NaN,NaN,binance-smart-chain,-0.003276,binance-smart-chain
trufin-staked-matic,NaN,NaN,NaN,NaN,NaN,NaN,-0.008518,polygon-pos,-0.008518,polygon-pos


In [4]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain
wrapped-steth,wstETH,2.802092e+10,0.150000,9830,0x5979d7b546e38e414f7e9822514be443a4800529,arbitrum-one
wrapped-eeth,WEETH,2.168628e+09,0.150000,9831,0x35751007a407ca6feffe80b3cb397736d2cf4dbe,arbitrum-one
rocket-pool-eth,rETH,1.712087e+09,0.150000,9831,0xec70dcb4a1efa46b8f2d97c310c9c4790ba5ffa8,arbitrum-one
renzo-restaked-eth,ezETH,1.259738e+09,0.150000,9831,0x2416092f143378750bb29b79ed961ab195cceea5,arbitrum-one
kelp-dao-restaked-eth,RSETH,7.256328e+08,0.136531,8947,0x4186BFC76E2E237523CBC30FD220FE055156b41F,arbitrum-one
coinbase-wrapped-staked-eth,CBETH,5.585275e+08,0.105089,6887,0x1debd73e752beaf79865fd6446b0c970eae7732f,arbitrum-one
staked-frax-ether,sfrxETH,4.855560e+08,0.091359,5987,0x95ab45875cffdba1e5f451b950bc2e42c0053f39,arbitrum-one
benqi-liquid-staked-avax,sAVAX,2.547688e+08,0.047936,3141,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,avalanche
stader-maticx,MATICX,1.014286e+08,0.019084,1250,0xfa68fb4628dff1028cfec22b4162fccd0d45efb6,polygon-pos
